Ստորև գրված առաջադրանքները կատարելու վերջնաժամկետը **15.05.2023 19:59** է։ Առաջադրանքները կատարելուց հետո, պետք է 

1. ներբեռնեք այս ֆայլը ձեր համակարգիչ (`File` $\to$ `Download .ipynb`)
2. վերնագրին ավելացնեք ձեր անուն, ազգանունը լատինատառ, օրինակ՝ *HW4_Kaggle_NshanPotikyan.ipynb* ֆորմատով
3. ուղարկեք ֆայլը **nshan.potikyan@gmail.com** հասցեին` նամակի թեմա (subject) դաշտում գրելով **HA4**

**Ուշադրություն**․ 
1. Վերոնշյալ պայմաններին չբավարարելու դեպքում ձեր աշխատանքը չի գնահատվի։
2. Կրկնօրինակ լուծումների համար կստանաք 0 միավոր։


# Kaggle մրցույթ «Մաշտոց» հավաքածուի վրա

Այս տնային առաջադրանքի նպատակը PyTorch գրադարանի օգնությամբ հայերեն ձեռագիր տառերը ճանաչող նեյրոնային ցանց ստեղծելն է։ 
Այդ նպատակին հասնելու համար պետք է իրականացնել հետևյալ քայլերը․

* Այցելեք [մրցույթի էջը](https://www.kaggle.com/competitions/mashtots-dataset-v2/overview) և սեղմեք "Join Competition"

* Ձեզնից կպահանջվի գրանցվել kaggle հարթակում, եթե դեռ չունեք գործող հաշիվ

* Գրանցվելուց հետո պետք է ընդունեք մրցույթի կանոնները սեղմելով "I understand and accept" 

* "Data" կոչվող հատվածում կարող եք ծանոթանալ տվյալների հետ և ներբեռնել դրանք ձեր համակարգիչ կամ կարող եք օգտվել kaggle notebook-ից (շատ ավելի հարմար տարբերակ) և տվյալները ավելացնեք այնտեղ։ Վերջինիս մասին կարող եք կարդալ ավելին [այստեղ](https://www.kaggle.com/docs/notebooks):

* Եթե չեք ուզում օգտվել kaggle notebook-ից, կարող եք վերբեռնել տվյալները Google Drive, և օգտագործել դրանք Colab միջավայրում Google Drive-ը mount անելուց հետո։

  ```python
  from google.colab import drive
  drive.mount('/content/drive')
  ```
* Տվյալներին հասանելիություն ունենալուց հետո կարող եք սկսել տվյալների մշակման և ալգորիթմի ուսուցման գործընթացը։ Ստորև կարող եք գտնել խնդրի լուծման համար որոշակի հուշումներ։

## Տվյալների մշակում

Եթե նկատեցիք, ուսուցողական բազմությունը իրենից ներկայացնում է նկարներ, որոնք գտնվում են համապատասխան տառերի պանակներում։ Կարող եք ստեղծել DataLoader, որի միջոցով այդ նկարները կարելի է կարդալ և տրամադրել մոդելին համապատասխան պիտակի հետ միասին։

In [ ]:
! pip install kaggle

In [ ]:
! mkdir ~/.kaggle


In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle competitions download mashtots-dataset-v2

In [ ]:
! unzip mashtots-dataset-v2.zip

In [ ]:
!pip install opencv-python

In [ ]:
!pip install torchvision

In [ ]:
!pip install torch

In [ ]:
import pandas as pd
import numpy as np
import torch
import os
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
import cv2
from torch.utils.data import DataLoader
import re

In [ ]:
pattern = re.compile(r'\d+')

# Define a function to extract the integer value from a filename
def get_numeric_value(filename):
    match = re.search(pattern, filename)
    if match:
        return int(match.group())
    else:
        return 0

In [ ]:
data_dir = '/content/Train/Train'
classes = os.listdir(data_dir)
int_list = [int(i) for i in classes]
sorted_list = sorted(int_list)
classes = [str(i) for i in sorted_list]
file_list = []
labels = []
for i, class_name in enumerate(classes):
    class_dir = os.path.join(data_dir, class_name)
    file_names = os.listdir(class_dir)
    file_names = sorted(file_names, key=get_numeric_value)
    file_paths = [os.path.join(class_dir, file_name) for file_name in file_names]
    file_list.extend(file_paths)
    labels.extend([i]*len(file_names))


In [ ]:
data = {"labels": labels, "image": file_list}
df = pd.DataFrame(data)
df

In [ ]:
from PIL import Image
from torch.utils.data import Dataset
from torchvision.transforms import transforms

class ImageDataset(Dataset):
    def __init__(self, data, transform=None):
        self.image = data['image']
        self.labels = pd.to_numeric(data['labels'])
        self.transform = transform

    def __len__(self):
        return len(self.image)

    def __getitem__(self, idx):
        img = Image.open(self.image[idx]).convert('RGB')
        label = self.labels[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

In [ ]:
from torch.utils.data import DataLoader
train_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                             std=[0.229, 0.224, 0.225])
    ])
image_dataset = ImageDataset(df, transform=train_transform)

## Վավերացման (Validation) բազմության ստեղծում

Քանի որ դուք չունեք **պիտակավորված** ստուգողական տվյալներ, ալգորիթմների որակը գնահատելու համար ցանկալի է, որ ստեղծեք առանձին վավերացման բազմություն ձեզ տրված ուսուցողական բազմությունից։ Ինքներդ կարող եք որոշել, թե որքան մասն եք պահելու ուսուցման և ալգորիթմի վավերացման համար։ 

In [ ]:
from torch.utils.data import DataLoader, random_split

# Split the dataset into training and validation sets
train_size = int(len(image_dataset) * 0.95)
val_size = len(image_dataset) - train_size
train_dataset, val_dataset = random_split(image_dataset, [train_size, val_size])

# Create DataLoader objects for the training and validation sets
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)

## Մոդելավորում

Այս փուլում ցանկալի է որ իրականացնեք տարբեր փորձեր, որպեսզի ստանաք մրցունակ ալգորիթմ։

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn,
               device: torch.device='cuda'):
    train_loss, train_acc = 0, 0
    model.to(device)
    for batch, (X, y) in enumerate(data_loader):
        # Send data to GPU
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss
        train_acc += accuracy_fn(y_true=y,
                                 y_pred=y_pred.argmax(dim=1)) # Go from logits -> pred labels

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

    # Calculate loss and accuracy per epoch and print out what's happening
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")

def test_step(data_loader: torch.utils.data.DataLoader,
              model: torch.nn.Module,
              loss_fn: torch.nn.Module,
              accuracy_fn,
              device: torch.device='cuda'):
    test_loss, test_acc = 0, 0
    model.to(device)
    model.eval() # put model in eval mode
    # Turn on inference context manager
    with torch.inference_mode(): 
        for X, y in data_loader:
            # Send data to GPU
            X, y = X.to(device), y.to(device)
            
            # 1. Forward pass
            test_pred = model(X)
            
            # 2. Calculate loss and accuracy
            test_loss += loss_fn(test_pred, y)
            test_acc += accuracy_fn(y_true=y,
                y_pred=test_pred.argmax(dim=1) # Go from logits -> pred labels
            )
        
        # Adjust metrics and print out
        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
        print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")

In [ ]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100 
    return acc

In [ ]:
import torch
import torchvision.models as models

net = models.resnet50(pretrained=True).to('cuda')
num_features = net.fc.in_features
net.fc = nn.Linear(num_features, 78)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)

In [ ]:
from tqdm.auto import tqdm
from timeit import default_timer as timer

torch.manual_seed(42)
train_time_start_on_gpu = timer()

epochs = 25
for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n---------")
    train_step(data_loader=train_loader, 
        model=net, 
        loss_fn=criterion,
        optimizer=optimizer,
        accuracy_fn=accuracy_fn
    )
    test_step(data_loader=val_loader,
        model=net,
        loss_fn=criterion,
        accuracy_fn=accuracy_fn
    )



## Կանխատեսումների հավաքագրում

Երբ կունենաք աշխատող ալգորիթմ, կարող եք փորձարկել այն kaggle միջավայրում՝ այնտեղ առկա new_test.csv տվյալների վրա (իրական պիտակները դուք չեք կարող տեսնել, ձեզ երևալու է միայն վերջնական արդյունքը [Categorization Accuracy](https://miro.medium.com/v2/resize:fit:373/1*yRa2inzTnyASJOre93ep3g.gif))։ Ստորև pseudo-code-ը կարող եք օգտագործել մոդելի կանխատեսումները kaggle հարթակ վերբեռնելու ձևաչափով պահպանելու համար։

In [ ]:
import pandas as pd

test = pd.read_csv('new_test/new_test.csv')
test.head()

In [ ]:
test_transform = transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                             std=[0.229, 0.224, 0.225])

In [ ]:
test_tensor = torch.tensor(test.values, dtype=torch.float) / 255
test_images = test_tensor.reshape((-1, 64, 64))
test_loader = DataLoader(test_images, batch_size=64)


In [ ]:
import csv

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.eval()
predictions = np.array([]) 
for minibatch in test_loader:
    minibatch = minibatch.to(device)
    minibatch = minibatch.unsqueeze(1)
    minibatch = minibatch.repeat(1, 3, 1, 1)
    minibatch = test_transform(minibatch)
    output = net(minibatch)
    pred = output.argmax(axis=1)
    predictions = np.append(predictions, pred.cpu().numpy())

In [ ]:
 predictions = predictions.astype(int)

In [ ]:
submission_df = pd.DataFrame({'Id': np.arange(1, test_images.shape[0]+1),
                      'Category': predictions})
submission_df.to_csv('my_submission.csv', index=False)

* Վերբեռնել ստեղծված ``my_submission.csv``֊ն Kaggle հարթակ՝ սեղմելով "Submit predictions" և կատարելով այդ էջում նշված քայլերը. 

* Ձեզնից կպահանձվի բնութագրել ձեր լուծումը մի քանի բառով "Describe Submission" բաժնում 

* Յուրաքանչյուր վերբեռնումից հետո կատարել էկրանանկար (screenshot), որտեղ հնարավոր կլինի տեսնել ալգորիթմի բնութագիրը և ստացված արդյունքը

* **Ստացված բոլոր էկրանանկարները պետք է ուղարկեք ձեր աշխատանքային notebook-ի հետ միասին ․zip ֆայլով՝ *HW4_Kaggle_NshanPotikyan.zip**

* Ամեն օր կարող եք կատարել առավելագույնը 5 վերբեռնում

* Մրցույթին խմբերով մասնակցել չի՛ կարելի

# Գնահատման կարգ

* Գնահատվելու համար ձեզնից կպահանջվի կատարել առնվազն 5 վերբեռնում 40% ճշգրտությունը գերազանցող մոդելներով։

* Նրանք ովքեր կստանան 90% և ավել ճշգրտություն ապահովող մոդելներ, կստանան 1/20 հավելյալ միավոր։

* Ամենաբարձր ճշգրտությամբ մոդելը ստացած ուսանողը կստանա 2/20 հավելյալ միավոր

* Ընդհանուր մրցույթում առաջին տեղը զբաղեցնելու դեպքում կստանաք 3/20 միավոր։